In [0]:
## Imports
import torch

import os
import numpy as np
import pandas as pd
import pandas.util.testing as tm
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim

import torch.nn.functional as F


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import sys


In [0]:
n_agents = 5
n_samples = 3000
n_train_samples =1500
use_cuda = torch.cuda.is_available()


In [0]:
### Data Representation of Moulin net
### Example of a data samples
#### tensor([ 1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  0.2848,  
####          1.0000, -1.0000, 1.0000, -1.0000, -1.0000,  0.3309,  
#####         1.0000, -1.0000,  1.0000, -1.0000,  1.0000,  0.6133,  
#####         1.0000,  1.0000,  1.0000, -1.0000,  1.0000,  0.6381, 
####          1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  0.8180])

peaks = []
for i in range(n_agents):
  peak = np.random.rand(n_samples)
  peaks.append(peak)
peaks = np.array(peaks)

def getbinaryrepresentation(array):
  representation = np.full((n_agents), -1)
  for value in array:
    representation[value] = 1
  return representation

def generateDataSamples(peaks):
  samples = []
  for i in range(n_samples):
    datasample = []
    currentpeaks = peaks[:,i]
    currentorder = np.argsort(currentpeaks)
    sortedorder = currentpeaks[currentorder]
    for j in range(n_agents):
      representation = getbinaryrepresentation(currentorder[:j+1])
      datasample.extend(representation)
      datasample.append(sortedorder[j])
    samples.append(datasample)
  return torch.FloatTensor(samples)


moulin_net_data = generateDataSamples(peaks)



In [0]:
## Moulin net data - split in train and test
indices = np.random.permutation(moulin_net_data.shape[0])
training_idx, test_idx = indices[:n_train_samples], indices[n_train_samples:]
moulin_train_data, moulin_test_data = moulin_net_data[training_idx,:], moulin_net_data[test_idx,:]
print('moulin training shape', moulin_train_data.shape)
print('moulin testing shape', moulin_test_data.shape)

moulin training shape torch.Size([1500, 30])
moulin testing shape torch.Size([1500, 30])


In [0]:
### Data Representation of Regret net
## Example of data sample for regret net
## tensor([0.1636, 0.6789, 0.8631, 0.2081, 0.8195])

peaks = np.array([])
for i in range(n_agents):
  peak = np.random.rand(n_samples)
  if(len(peaks) == 0):
    peaks = peak
  else:
    peaks =  np.dstack((peaks,peak))

peaks = np.reshape(peaks, (n_samples, n_agents ))
regret_net_data = torch.FloatTensor(peaks)


In [0]:
## Regret net data - split in train and test
indices = np.random.permutation(regret_net_data.shape[0])
training_idx, test_idx = indices[:n_train_samples], indices[n_train_samples:]
regretnet_train_data, regretnet_test_data = regret_net_data[training_idx,:], regret_net_data[test_idx,:]
print('regret training shape', regretnet_train_data.shape)
print('regret testing shape', regretnet_test_data.shape)

regret training shape torch.Size([1500, 5])
regret testing shape torch.Size([1500, 5])


In [0]:
class MoulinNet(nn.Module):
    def __init__(self, n_agents, K):
        super(MoulinNet, self).__init__()
        self.n_agents = n_agents
        self.K = K
        self.L = 3
        self.J = 3
        self.datasamplelength = (n_agents+1)*n_agents
        ## if there are K facility, replicate the network K times
        self.asNN = nn.ModuleList([ nn.Linear(self.n_agents,self.L*self.J)   for i in range(K)])
        
    def forward(self, x):
        ## Outputvalues will contain the location of all K facilities for each data samples
        outputvalues = torch.tensor([])
        for k in range(self.K):
          X_clone = x.clone()

          ## Passing through a_s  neural network 
          ## from s get the a_s value
          ## looping for each s value i.e. where number of s values are equal to no of agents
          for i in range(0, self.datasamplelength  ,  self.n_agents+1  ):
            ## layer 1 :  Pass through NN 
            tempvalue = self.asNN[k](X_clone[:,i:i+self.n_agents].clone())
            ## layer 2 : get maximum 
            for j in range(0, self.L*self.J,self.J):
              maxvalue_as, indicies = tempvalue[:,j:j+self.J].max(axis=1)
              maxvalue_as = maxvalue_as.unsqueeze_(-1)
              tempvalue[:,j:j+self.J] = maxvalue_as.clone()   
            ## layer 3 : get minimum
            minvalue_as, indicies = tempvalue.min(axis=1)
            minvalue_as = minvalue_as.unsqueeze_(-1)

            ### replacing the X_clone value with the minvalue obtained for s
            ### Before : 
            X_clone[:,i:i+self.n_agents] = minvalue_as.clone()
            ### After : 

          ## overall X_clone[0] will look like 


          ## Take maximum of a_s and peak
          for i in range(0, (self.n_agents+1)*self.n_agents  ,  self.n_agents+1  ):
            maxvalue, indicies = X_clone[:,i:i+self.n_agents+1].max(axis=1)
            maxvalue = maxvalue.unsqueeze_(-1)
            X_clone[:,i:i+self.n_agents+1] = maxvalue.clone()

          ## from all that is left, take min and that is the location we obtain
          minvalue, indicies = X_clone.min(axis=1)
          minvalue = minvalue.unsqueeze_(-1)

          ## for each K, append it to outputvalues
          outputvalues  = torch.cat([outputvalues, minvalue.clone()], axis=1)
        return outputvalues

    def loss_function(self, y, x):
      loss = torch.tensor([])
      ## For each facility
      for k in range(self.K):
        ## fetch localtion for kth facility for all data points      
        location_of_k_facility = y[:,k:k+1]
        ## calculate loss i.e. peak - location
        ### for now we are using x as whole to substract, later we will make all entries 0 expect the peak ones
        appendloss = torch.add(x,-location_of_k_facility)    
        ### append to loss for each facility
        loss  = torch.cat([loss,appendloss ], axis = 1)

      loss = torch.abs(loss)
      loss = torch.reshape(loss, (x.shape[0],self.K,self.datasamplelength) )

      ## for each each agent, we choose the minimum loss , i.e. loss corresponding to the location closest to its peak
      loss , indices = torch.min(loss, axis=1)

      ## all the entries expected peaks => 0      
      for i in range(0,(self.n_agents+1)*self.n_agents, self.n_agents+1 ):
        loss[:,i : i+self.n_agents] = 0  
      
      ## loss will be mean over all data sample, and for each data sample , mean over n_agents
      loss = torch.mean((torch.sum(  loss , dim=1))/self.n_agents)
      return loss




In [0]:
## 26 May 20
## multi facility
class RegretNet(nn.Module):
    def __init__(self, n_agents,K):
        super(RegretNet, self).__init__()
        self.n_agents = n_agents
        self.unitperlayer = 40
        self.K = K
        self.M = 5  ## no of betas to calculate misreported peaks
        self.fc1 = nn.Linear(self.n_agents,self.unitperlayer)
        self.fc2 = nn.Linear(self.unitperlayer,self.unitperlayer)
        self.fc3 = nn.Linear(self.unitperlayer,self.unitperlayer)
        self.fc4 = nn.Linear(self.unitperlayer,self.unitperlayer)
        self.fc5 = nn.Linear(self.unitperlayer,self.K)
        torch.nn.init.uniform_(self.fc1.weight, a=0.0, b=0.01)
        self.fc1.bias.data.fill_(0.1)
        torch.nn.init.uniform_(self.fc2.weight, a=0.0, b=0.01)
        self.fc2.bias.data.fill_(0.1)
        torch.nn.init.uniform_(self.fc3.weight, a=0.0, b=0.01)
        self.fc3.bias.data.fill_(0.1)
        torch.nn.init.uniform_(self.fc4.weight, a=0.0, b=0.01)
        self.fc4.bias.data.fill_(0.1)
        torch.nn.init.uniform_(self.fc5.weight, a=0.0, b=0.01)
        self.fc5.bias.data.fill_(0.1)

    def forward(self, x):
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = F.relu(self.fc3(x))
      x = F.relu(self.fc4(x))
      x = F.relu(self.fc5(x))
      return x

    def calculateUtility(self,peak_arr, output):
        ## for agent i, peaks of all datasampes
        ## peak_arr :  n_samples x 1
        ### output : n_samples x K
        loss = torch.tensor([])
        if use_cuda:
          loss = loss.cuda()

        ## For each facility calcuate loss
        for k in range(self.K):        
          k_facility_location = output[k:k+1]
          appendloss = torch.add(peak_arr,-k_facility_location)    
          loss  = torch.cat([loss,appendloss ], axis = 0)
        loss = torch.abs(loss)
        loss = torch.reshape(loss, (self.K,self.n_agents) )
        ## take minimum loss amoung all k facility i.e. loss corresponding to closest facility to peak
        loss , indices = torch.min(loss, axis=0)
        utility = -loss
        return utility
    
    def calculateRegret(self,y,x, betas):
      ## y : is output location of K facility for all n_samples  n_samples x K
      ### x is n_samples x n_agents (peaks of data)
      ### betas - that are sampled to calculate regret
      ### we will calculate regret only if a peak matches any of betas, and then misreported peaks becomes rest of the betas
      ## Examples if betas is [0.2 , 0.4, 0.6, 0.8 , 1]
      ## and peak is [0.4] then you go ahead and find regret with misreported values [0.2,0.6,0.8,1]
      ## For practicality, we keep a threshold , instead of fully matching betas and peaks

      ## From the paper 
      ### For each example j, (5) computes the maximum utility agent i could gain if her true peak was one
      ### of β1;...; βM and she chose to misreport it as one of β1,..., βM.

      regret = []
      threshold = 0.01
      ## For each agent
      for i in range(self.n_agents):
        ## fetch their peaks
        peak_of_agent_i = x[ :, i:i+1]
        count = 0
        regret_for_agent_i = []

        ## For each peak, 
        for j, peak in enumerate(peak_of_agent_i):
          valuedifference = torch.add(betas, -peak)
          isValuePresentWithThreshold = torch.abs(valuedifference) <= threshold
          ## Find if peak matches any value in betas
          if True in isValuePresentWithThreshold:
            count += 1
            index =  torch.where(isValuePresentWithThreshold == True)
            ## to calucate regret
            regret_for_an_sample = []

            ## calculate regret for all other values of betas 
            for b in range(self.M):
              if( b != index[0][0]):
                datasample = x[j].clone()
                
                ## misreported output location Kx1
                misreporteddatasample = datasample.clone()
                misreporteddatasample[i] = betas[b]
                misrerpoted_output = self.forward(misreporteddatasample)
                
                ## acutal output location K x 1
                actualdatasample = datasample.clone()
                actualdatasample[i] = betas[index[0][0]]
                actualdata_output = self.forward(actualdatasample)

                ## utility will be calcuated considering the actual peak 1x1
                misreported_utility = self.calculateUtility(actualdatasample,misrerpoted_output)
                actual_utility = self.calculateUtility(actualdatasample,actualdata_output)

                difference_for_agent_i = misreported_utility[i] -   actual_utility[i]
                regret_for_an_sample.append(difference_for_agent_i)

            regret_for_an_sample = torch.FloatTensor(regret_for_an_sample)
            ## for all the misreported peaks, chose the maximum regret for that sample
            regret_for_an_sample = torch.max(regret_for_an_sample)
            ## append it to the regret of agent 
            regret_for_agent_i.append(regret_for_an_sample)
          else:
            ## when peaks doesnt matches any betas, we simply append 0 peak
            regret_for_agent_i.append(0)
        

        regret_for_agent_i = torch.FloatTensor(regret_for_agent_i)
        # regret_for_an_sample = torch.mean(regret_for_an_sample)
        # taking mean over all samples for agent and append to regret
        regret_for_agent_i = torch.mean(regret_for_agent_i)
        regret.append(regret_for_agent_i)
        print('For agent ' , i , ' no of beta matches with peak', count)
      print('regret is ', regret)

      regret = torch.FloatTensor(regret)
      maxregretamoungagents = torch.max(regret)
      print('max regret amoung agents', maxregretamoungagents)
      return regret, maxregretamoungagents



    def loss_function(self, y, x, _lamdda , _rho):

      ## Betas for calculating regret
      betas =  np.random.rand(self.M)
      betas = torch.FloatTensor(betas)
      if use_cuda:
        betas = betas.cuda()
      print('betas', betas)

      regret , maxregretamoungagents = self.calculateRegret(y,x,betas)

      ## calculatting social cost the same way we did in moulin net
      loss = torch.tensor([]).cuda()
      for k in range(self.K):        
        loss_for_k_facility = y[:,k:k+1]
        appendloss = torch.add(x,-loss_for_k_facility)    
        loss  = torch.cat([loss,appendloss ], axis = 1)
      loss = torch.abs(loss)
      loss = torch.reshape(loss, (x.shape[0],self.K,self.n_agents) )
      loss , indices = torch.min(loss, axis=1)
      loss = torch.mean((torch.sum(  loss , dim=1))/self.n_agents)

      print('social cost ' , loss)

      loss = loss + (maxregretamoungagents * _lamdda) +  ( maxregretamoungagents**2 ) * _rho
      return loss, maxregretamoungagents




In [0]:
### Train and Testing of Moulin Net

K = 1
net = MoulinNet(n_agents,K)
epochs = 40
learning_rate = 0.1

def moulin_train(X_train,epochs=epochs, learning_rate=learning_rate):
  optimizer = optim.Adam(net.parameters(), lr= learning_rate)
  with torch.autograd.set_detect_anomaly(True):
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = net(X_train)
        train_loss = net.loss_function(y_pred,X_train)    
        print( 'Epoch : ', epoch ,  'training loss ', train_loss)
        train_loss.backward()
        optimizer.step()

def moulin_test(X_test):
    y_test = net(X_test)
    test_loss = net.loss_function(y_test,X_test)    
    print('Testing Loss is ', test_loss)


moulin_train(moulin_train_data)
moulin_test(moulin_test_data)


Epoch :  0 training loss  tensor(0.3000, grad_fn=<MeanBackward0>)
Epoch :  1 training loss  tensor(0.2614, grad_fn=<MeanBackward0>)
Epoch :  2 training loss  tensor(0.2434, grad_fn=<MeanBackward0>)
Epoch :  3 training loss  tensor(0.2357, grad_fn=<MeanBackward0>)
Epoch :  4 training loss  tensor(0.2329, grad_fn=<MeanBackward0>)
Epoch :  5 training loss  tensor(0.2293, grad_fn=<MeanBackward0>)
Epoch :  6 training loss  tensor(0.2269, grad_fn=<MeanBackward0>)
Epoch :  7 training loss  tensor(0.2165, grad_fn=<MeanBackward0>)
Epoch :  8 training loss  tensor(0.2056, grad_fn=<MeanBackward0>)
Epoch :  9 training loss  tensor(0.2042, grad_fn=<MeanBackward0>)
Epoch :  10 training loss  tensor(0.2039, grad_fn=<MeanBackward0>)
Epoch :  11 training loss  tensor(0.2034, grad_fn=<MeanBackward0>)
Epoch :  12 training loss  tensor(0.2028, grad_fn=<MeanBackward0>)
Epoch :  13 training loss  tensor(0.2017, grad_fn=<MeanBackward0>)
Epoch :  14 training loss  tensor(0.2005, grad_fn=<MeanBackward0>)
Epoch

In [0]:
### Train and Test Regret net
### mini batch training
K = 1
mini_batch_size  = 500
lr = 0.005
epochs = 40
_rho = 0.0005


net = RegretNet(n_agents,K)
if use_cuda:
    net = net.cuda()

def create_mini_batches(data, batch_size): 
    mini_batches = [] 
    indices = np.random.permutation(data.shape[0])
    n_minibatches = data.shape[0] // batch_size   
    for i in range(n_minibatches ): 
        mini_batch_idx= indices[i * batch_size:(i + 1)*batch_size]
        mini_batch = data[ mini_batch_idx, :] 
        mini_batches.append(mini_batch) 
    return mini_batches 


## decay learning rate
def adjust_learning_rate(optimizer, epoch,  no_iterations, quantity, lr= lr):
    lr = lr * (quantity ** (epoch // no_iterations))
    for param_group in optimizer.param_groups:
      param_group['lr'] = lr
      print('current learning rate ', lr)


def regretnet_train(X_train,epochs=epochs, learning_rate=lr, ):

  _lamdda = 0.1
  optimizer = optim.Adam(net.parameters(), lr= learning_rate)
  with torch.autograd.set_detect_anomaly(True):
    for epoch in range(epochs):
        totalerror = []
        minibatches = create_mini_batches(X_train,mini_batch_size )
        for minibatch in minibatches:
          optimizer.zero_grad()
          ## decay lr after some epochs
          adjust_learning_rate(optimizer, epoch, 10, 0.99)

          y_pred = net(minibatch)
          train_loss , maxregret = net.loss_function(y_pred,minibatch,_lamdda,_rho)    

          ## total error for a epoch
          totalerror.append(train_loss)

          train_loss.backward()
          optimizer.step()

          print('Epoch : ', epoch , 'minibatch loss  ', train_loss)
        totalerror = torch.FloatTensor(totalerror)
        print( 'Epoch : ', epoch ,  'training loss ', torch.mean(totalerror))

        ## Update rule for lambda
        if(epoch % 5 == 0):
          print(' current lambda', _lamdda , ' to minus ',  _rho* maxregret)
          _lamdda = _lamdda - _rho* maxregret
          print(' _lamdda value ', _lamdda)
  return _lamdda        



def regretnet_test(X_test):
    y_test = net(X_test)
    test_loss, maxregret = net.loss_function(y_test,X_test,_lamdda,_rho)
    print('Testing Loss is ', test_loss)
    print('max regret is ', maxregret)


_lamdda = regretnet_train(regretnet_train_data.cuda())
regretnet_test(regretnet_test_data.cuda())
# 

current learning rate  0.005
betas tensor([0.3734, 0.2209, 0.6043, 0.5474, 0.3748], device='cuda:0')
For agent  0  no of beta matches with peak 40
For agent  1  no of beta matches with peak 41
For agent  2  no of beta matches with peak 38
For agent  3  no of beta matches with peak 32
For agent  4  no of beta matches with peak 41
regret is  [tensor(8.6188e-08), tensor(8.3655e-08), tensor(1.2296e-07), tensor(7.7814e-08), tensor(1.4436e-07)]
max regret amoung agents tensor(1.4436e-07)
social cost  tensor(0.3997, device='cuda:0', grad_fn=<MeanBackward0>)
Epoch :  0 minibatch loss   tensor(0.3997, device='cuda:0', grad_fn=<AddBackward0>)
current learning rate  0.005
betas tensor([0.7304, 0.9506, 0.6830, 0.9985, 0.1749], device='cuda:0')
For agent  0  no of beta matches with peak 39
For agent  1  no of beta matches with peak 34
For agent  2  no of beta matches with peak 61
For agent  3  no of beta matches with peak 50
For agent  4  no of beta matches with peak 45
regret is  [tensor(1.1557e-0